In [1]:
import os
import boto3

#1. s3에서 결과 파일 다운받기 

In [2]:
BUCKET_NAME = ''
s3_client = boto3.client('s3')

In [3]:
model_info={"alexnet" : 233.1,
            "efficientnet_b0":20.5,
            "inception_v3":104,
            "mnasnet": 17,
            "mobilenet_v2":13.6,
            "resnet18":44.7,
            "resnet50":97.8,
            "shufflenet":8.9,
            "squeezenet":4.8,
            "vgg16":527.8}
model_name = list(model_info.keys())
hardware_list=["arm","intel"]
optimize_list=["base","onnx","tvm"]
lambda_memory_list=[512,1024,2048,4096,8192,10240]

In [45]:
# lambda_memory=10240
batchsize=1

In [46]:
for lambda_memory in lambda_memory_list:
    for hardware in hardware_list:
        for optimize in optimize_list:
            for model in model_name:
                try:
                    os.makedirs(os.path.dirname(f'./exp2/{lambda_memory}/{optimize}_{hardware}/'), exist_ok=True)
                    s3_client.download_file(BUCKET_NAME, 
                                        f'results/{optimize}/{hardware}/{model}_{model_info[model]}_{batchsize}_{lambda_memory}.json',
                                        f'./exp2/{lambda_memory}/{optimize}_{hardware}/{model}_{model_info[model]}_{batchsize}.json')
                except:
                    print(f"Not Exist in S3 :'results/{optimize}/{hardware}/{model}_{model_info[model]}_{batchsize}_{lambda_memory}.json', ")

Not Exist in S3 :'results/base/arm/vgg16_527.8_1_512.json', 
Not Exist in S3 :'results/onnx/arm/alexnet_233.1_1_512.json', 
Not Exist in S3 :'results/onnx/arm/vgg16_527.8_1_512.json', 
Not Exist in S3 :'results/tvm/arm/alexnet_233.1_1_512.json', 
Not Exist in S3 :'results/tvm/arm/vgg16_527.8_1_512.json', 
Not Exist in S3 :'results/base/intel/vgg16_527.8_1_512.json', 
Not Exist in S3 :'results/onnx/intel/alexnet_233.1_1_512.json', 
Not Exist in S3 :'results/onnx/intel/vgg16_527.8_1_512.json', 
Not Exist in S3 :'results/tvm/intel/alexnet_233.1_1_512.json', 
Not Exist in S3 :'results/tvm/intel/resnet18_44.7_1_512.json', 
Not Exist in S3 :'results/tvm/intel/resnet50_97.8_1_512.json', 
Not Exist in S3 :'results/tvm/intel/vgg16_527.8_1_512.json', 
Not Exist in S3 :'results/onnx/arm/vgg16_527.8_1_1024.json', 
Not Exist in S3 :'results/tvm/arm/vgg16_527.8_1_1024.json', 
Not Exist in S3 :'results/onnx/intel/vgg16_527.8_1_1024.json', 
Not Exist in S3 :'results/tvm/intel/vgg16_527.8_1_1024.json',

# json to pandas

In [47]:
import json
import pandas as pd

total=pd.DataFrame()
for lambda_memory in lambda_memory_list:
    for hardware in hardware_list:
        for optimize in optimize_list:
            for model in model_name:
                try:
                    with open(f'./exp2/{lambda_memory}/{optimize}_{hardware}/{model}_{model_info[model]}_{batchsize}.json') as f:
                        js = json.loads(f.read()) ## json 라이브러리 이용
                        df = pd.DataFrame([js])
                    total = pd.concat([total,df])
                except:
                    continue

In [48]:
total = total.reset_index(drop=True)

In [49]:
total

,model_name,model_size,hardware,framework,optimizer,lambda_memory,batchsize,convert_time,inference_time,user_email,request_id,log_group_name,max_memory_used
0,alexnet,233.1,arm,torch,base,512,1,0.000000,0.599954,subean@kookmin.ac.kr,665cfc5c-6da8-458f-a6b5-e9f4873ef2c8,/aws/lambda/serving_arm_torch_512,512
1,efficientnet_b0,20.5,arm,torch,base,512,1,0.000000,1.560323,subean@kookmin.ac.kr,3de8f830-b673-4ade-bcd1-07464c041a95,/aws/lambda/serving_arm_torch_512,366
2,inception_v3,104,arm,torch,base,512,1,0.000000,2.360193,subean@kookmin.ac.kr,0af106eb-e961-41c8-83a6-06a3345c264d,/aws/lambda/serving_arm_torch_512,512
3,mnasnet,17,arm,torch,base,512,1,0.000000,0.941607,subean@kookmin.ac.kr,6346797a-3cd0-4783-ba04-2315c42b993d,/aws/lambda/serving_arm_torch_512,512
4,mobilenet_v2,13.6,arm,torch,base,512,1,0.000000,0.838275,subean@kookmin.ac.kr,dc580c0a-8889-4ba1-bb72-77f9fc546554,/aws/lambda/serving_arm_torch_512,339
...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,resnet18,44.7,intel,torch,tvm,10240,1,19.321746,0.035605,subean@kookmin.ac.kr,3f11e815-b09c-4ef6-bc47-18474cd53230,/aws/lambda/serving_intel_tvm_10240,1157
336,resnet50,97.8,intel,torch,tvm,10240,1,46.081084,0.090846,subean@kookmin.ac.kr,405e846d-a4db-4bec-9b5b-21f7b5ef821c,/aws/lambda/serving_intel_tvm_10240,699
337,shufflenet,8.9,intel,torch,tvm,10240,1,54.177398,0.012504,subean@kookmin.ac.kr,370410bf-cca2-4286-b691-10f0dae0b6fc,/aws/lambda/serving_intel_tvm_10240,699
338,squeezenet,4.8,intel,torch,tvm,10240,1,12.999760,0.022008,subean@kookmin.ac.kr,518082c9-e399-4944-bd6e-62debd8da9a5,/aws/lambda/serving_intel_tvm_10240,1157


In [50]:
total.to_csv(f"./total_df_{batchsize}.csv")

In [53]:
batchsize_list = [1,2,4,8,16,32]
all_df = pd.DataFrame()
for batchsize in batchsize_list:
    data = pd.read_csv(f"./total_df_{batchsize}.csv")
    all_df =  pd.concat([all_df,data])

In [60]:
all_df

,model_name,model_size,hardware,framework,optimizer,lambda_memory,batchsize,convert_time,inference_time,user_email,request_id,log_group_name,max_memory_used
0,alexnet,233.1,arm,torch,base,512,1,0.000000,0.599954,subean@kookmin.ac.kr,665cfc5c-6da8-458f-a6b5-e9f4873ef2c8,/aws/lambda/serving_arm_torch_512,512
1,efficientnet_b0,20.5,arm,torch,base,512,1,0.000000,1.560323,subean@kookmin.ac.kr,3de8f830-b673-4ade-bcd1-07464c041a95,/aws/lambda/serving_arm_torch_512,366
2,inception_v3,104.0,arm,torch,base,512,1,0.000000,2.360193,subean@kookmin.ac.kr,0af106eb-e961-41c8-83a6-06a3345c264d,/aws/lambda/serving_arm_torch_512,512
3,mnasnet,17.0,arm,torch,base,512,1,0.000000,0.941607,subean@kookmin.ac.kr,6346797a-3cd0-4783-ba04-2315c42b993d,/aws/lambda/serving_arm_torch_512,512
4,mobilenet_v2,13.6,arm,torch,base,512,1,0.000000,0.838275,subean@kookmin.ac.kr,dc580c0a-8889-4ba1-bb72-77f9fc546554,/aws/lambda/serving_arm_torch_512,339
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,resnet18,44.7,intel,torch,tvm,10240,32,20.336548,1.204782,subean@kookmin.ac.kr,6845c3d0-5a5c-4629-aa94-491b81f26188,/aws/lambda/serving_intel_tvm_10240,1279
277,resnet50,97.8,intel,torch,tvm,10240,32,46.331776,3.386071,subean@kookmin.ac.kr,f478a1d8-e8a6-47b6-b071-e68acfb626c8,/aws/lambda/serving_intel_tvm_10240,1161
278,shufflenet,8.9,intel,torch,tvm,10240,32,55.070966,0.276298,subean@kookmin.ac.kr,273fbeb2-7357-4c51-8d04-49a12e5ed5e9,/aws/lambda/serving_intel_tvm_10240,387
279,squeezenet,4.8,intel,torch,tvm,10240,32,13.048629,0.775123,subean@kookmin.ac.kr,a115d368-655e-4ec8-99ff-114ac5fd070b,/aws/lambda/serving_intel_tvm_10240,1378


In [61]:
all_df.to_csv(f"./all_batch_df.csv")